In [10]:
import qiskit
import numpy as np
from qiskit.quantum_info import Statevector, partial_trace
from music21 import stream, chord, instrument, volume
from qiskit.circuit.library import QFT
import random

# Chromatic scale notes (C4 to B4)
chromatic_notes = list(range(60, 72))  # MIDI notes for one octave chromatic scale

def chromatic_note_from_angle(angle, offset=0):
    if np.isnan(angle) or np.isinf(angle):
        base_note = chromatic_notes[0]
    else:
        normalized = angle % (2 * np.pi)
        index = int((normalized / (2 * np.pi)) * len(chromatic_notes)) % len(chromatic_notes)
        base_note = chromatic_notes[index]
    return base_note + offset

def duration_from_theta(theta, min_duration=0.25, max_duration=2.0):
    theta = np.clip(theta, 0, np.pi)
    dur = min_duration + (theta / np.pi) * (max_duration - min_duration)
    return dur

def volume_scheme(step, scheme, total_steps=20):
    min_vel, max_vel = 40, 127
    if scheme == 'crescendo':
        return int(min_vel + (step / (total_steps-1)) * (max_vel - min_vel))
    elif scheme == 'decrescendo':
        return int(max_vel - (step / (total_steps-1)) * (max_vel - min_vel))
    elif scheme == 'steady':
        return int((min_vel + max_vel) // 2)
    elif scheme == 'random':
        return np.random.randint(min_vel, max_vel+1)
    else:
        return int((min_vel + max_vel) // 2)

def qubit_state_to_notes_with_duration_and_volume(alpha, beta, offset=0, step=0, scheme='crescendo', total_steps=20):
    abs_alpha = np.clip(np.abs(alpha), 0, 1)
    theta = 2 * np.arccos(abs_alpha)
    phi = np.angle(beta) - np.angle(alpha)

    # Lower pitch by 24 semitones (2 octaves down)
    octave_lower = -24

    # Map phi to chromatic note pitch with offset per qubit (key shift + octave lowering)
    pitch_note = chromatic_note_from_angle(phi, offset + octave_lower)
    dur = duration_from_theta(theta)
    vel = volume_scheme(step, scheme, total_steps)

    c = chord.Chord([pitch_note, pitch_note], quarterLength=dur)
    c.volume = volume.Volume(velocity=vel)
    return c

def qubit_to_instrument(index):
    instruments = [
        instrument.Flute(), instrument.Piano(), instrument.Violin(), instrument.Guitar(),
        instrument.Clarinet(), instrument.Trumpet(), instrument.Bass(),
        instrument.Oboe(), instrument.Horn(), instrument.Bassoon()
    ]
    return instruments[index % len(instruments)]

def apply_gate(state, gate, qargs, num_qubits):
    temp_circuit = qiskit.QuantumCircuit(num_qubits)
    temp_circuit.append(gate, [temp_circuit.qubits.index(q) for q in qargs])
    return state.evolve(temp_circuit)

def measure_coin(state, coin_qubit):
    probs = state.probabilities_dict()
    keys = list(probs.keys())
    probabilities = np.array([probs[k] for k in keys])
    measured_outcome = np.random.choice(keys, p=probabilities)
    coin_bit = int(measured_outcome[-1 - coin_qubit])
    proj_vec = np.array([1 if k[-1 - coin_qubit] == str(coin_bit) else 0 for k in keys])
    amplitudes = np.array([state.data[int(k, 2)] for k in keys])
    projected_data = amplitudes * proj_vec
    norm = np.linalg.norm(projected_data)
    if norm > 0:
        projected_data /= norm
    new_state_data = np.zeros_like(state.data)
    for i, k in enumerate(keys):
        new_state_data[int(k, 2)] = projected_data[i]
    return Statevector(new_state_data), coin_bit

def circuit_to_music_explicit_with_measure(circuit, coin_qubit=0, total_steps=20):
    num_qubits = circuit.num_qubits
    state = Statevector.from_label('0' * num_qubits)
    qubit_states = {i: [] for i in range(num_qubits)}
    music_score = stream.Score()
    
    # Use only crescendo volume scheme
    qubit_volume_schemes = ['crescendo'] * num_qubits
    
    step_counter = 0

    # Insert barriers randomly as measurement points
    barrier_positions = sorted(random.sample(range(len(circuit.data) + total_steps), total_steps))
    barrier_inserted = 0
    data_idx = 0

    for pos in range(len(circuit.data) + total_steps):
        if barrier_inserted < total_steps and pos == barrier_positions[barrier_inserted]:
            state, _ = measure_coin(state, coin_qubit)
            for i in range(num_qubits):
                reduced = partial_trace(state, [j for j in range(num_qubits) if j != i])
                alpha, beta = reduced.data[0][0], reduced.data[1][0]
                qubit_states[i].append((alpha, beta, step_counter))
            step_counter += 1
            barrier_inserted += 1
        else:
            if data_idx < len(circuit.data):
                inst, qargs, _ = circuit.data[data_idx]
                if inst.name != 'barrier':
                    state = apply_gate(state, inst, qargs, num_qubits)
                data_idx += 1

    for i in range(num_qubits):
        part = stream.Part()
        part.insert(0, qubit_to_instrument(i))
        offset = i * 1  # 1 semitone apart
        scheme = qubit_volume_schemes[i]
        for (alpha, beta, step) in qubit_states[i]:
            note_chord = qubit_state_to_notes_with_duration_and_volume(alpha, beta, offset, step, scheme, total_steps)
            part.append(note_chord)
        music_score.append(part)

    return music_score, qubit_states

### --- Grover's algorithm oracle and diffuser --- ###

def grover_oracle(qc, position_qubits, marked_element):
    n = len(position_qubits)
    binary_str = f"{marked_element:0{n}b}"
    # Apply X gates where bit is 0
    for i, bit in enumerate(binary_str[::-1]):
        if bit == '0':
            qc.x(position_qubits[i])
    # Multi-controlled Z using mcx (multi-controlled X) conjugated with H
    qc.h(position_qubits[-1])
    # Use mcx gate:
    if n > 1:
        qc.mcx(position_qubits[:-1], position_qubits[-1])
    else:
        qc.x(position_qubits[-1])  # if only one qubit, just X
    qc.h(position_qubits[-1])
    # Undo X gates
    for i, bit in enumerate(binary_str[::-1]):
        if bit == '0':
            qc.x(position_qubits[i])
    qc.barrier()

def grover_diffuser(qc, position_qubits):
    n = len(position_qubits)
    qc.h(position_qubits)
    qc.x(position_qubits)
    qc.h(position_qubits[-1])
    if n > 1:
        qc.mcx(position_qubits[:-1], position_qubits[-1])
    else:
        qc.x(position_qubits[-1])
    qc.h(position_qubits[-1])
    qc.x(position_qubits)
    qc.h(position_qubits)
    qc.barrier()

def run_grover_circuit(num_qubits, coin_qubit, position_qubits, ancillas, marked_element, num_iterations):
    qc = qiskit.QuantumCircuit(num_qubits)
    # Prepare uniform superposition on position qubits
    qc.h(position_qubits)
    qc.barrier()
    
    for _ in range(num_iterations):
        grover_oracle(qc, position_qubits, marked_element)
        grover_diffuser(qc, position_qubits)
    
    return qc

def run_qft_circuit(num_qubits, coin_qubit, position_qubits, ancillas):
    qc = qiskit.QuantumCircuit(num_qubits)
    # Initialize position qubits to superposition
    qc.h(position_qubits)
    qc.barrier()
    
    # Apply QFT on position qubits
    qft_circ = QFT(len(position_qubits), do_swaps=True)
    qc.append(qft_circ.to_gate(), position_qubits)
    qc.barrier()
    
    return qc


# Parameters
num_position_qubits = 9
num_ancillas = 8
num_qubits = 1 + num_position_qubits + num_ancillas  # 18 total qubits

coin_qubit = 0
position_qubits = list(range(1, 1 + num_position_qubits))
ancillas = list(range(1 + num_position_qubits, num_qubits))

# Grover parameters
marked_element = 5
num_grover_iterations = 6

qc_grover = run_grover_circuit(num_qubits, coin_qubit, position_qubits, ancillas, marked_element, num_grover_iterations)
music_score_grover, qubit_states_grover = circuit_to_music_explicit_with_measure(qc_grover, coin_qubit=coin_qubit, total_steps=num_grover_iterations*2)

qc_qft = run_qft_circuit(num_qubits, coin_qubit, position_qubits, ancillas)
music_score_qft, qubit_states_qft = circuit_to_music_explicit_with_measure(qc_qft, coin_qubit=coin_qubit, total_steps=10)

# Show textual music summary
print("Grover's algorithm music:")
music_score_grover.show('text')
music_score_grover.write('midi', fp='grover_music.mid')

print("\nQuantum Fourier Transform music:")
music_score_qft.show('text')
music_score_qft.write('midi', fp='qft_music.mid')

# Draw circuits to files
qc_grover.draw(output='mpl', fold=100).savefig('grover_circuit.png')
qc_qft.draw(output='mpl', fold=100).savefig('qft_circuit.png')


Grover's algorithm music:
{0.0} <music21.stream.Part 0x11837d4c0>
    {0.0} <music21.instrument.Flute 'Flute'>
    {0.0} <music21.chord.Chord C2 C2>
    {0.25} <music21.chord.Chord C2 C2>
    {0.5} <music21.chord.Chord C2 C2>
    {0.75} <music21.chord.Chord C2 C2>
    {1.0} <music21.chord.Chord C2 C2>
    {1.25} <music21.chord.Chord C2 C2>
    {1.5} <music21.chord.Chord C2 C2>
    {1.75} <music21.chord.Chord C2 C2>
    {2.0} <music21.chord.Chord C2 C2>
    {2.25} <music21.chord.Chord C2 C2>
    {2.5} <music21.chord.Chord C2 C2>
    {2.75} <music21.chord.Chord C2 C2>
{3.0} <music21.stream.Part 0x1174b8e60>
    {0.0} <music21.instrument.Piano 'Piano'>
    {0.0} <music21.chord.Chord C#2 C#2>
    {1.4167} <music21.chord.Chord G2 G2>
    {3.3994} <music21.chord.Chord C#2 C#2>
    {4.8456} <music21.chord.Chord G2 G2>
    {6.8072} <music21.chord.Chord C#2 C#2>
    {8.2815} <music21.chord.Chord C#2 C#2>
    {9.7916} <music21.chord.Chord G2 G2>
    {10.5224} <music21.chord.Chord G2 G2>
    {12.

In [15]:
import qiskit
import numpy as np
from qiskit.quantum_info import Statevector, partial_trace
from music21 import stream, chord, instrument, volume
from qiskit.circuit.library import Initialize

# Chromatic scale notes (C4 to B4, MIDI 60-71)
chromatic_notes = list(range(60, 72))  # One octave chromatic scale

def chromatic_note_from_angle(angle, offset=0):
    if np.isnan(angle) or np.isinf(angle):
        base_note = chromatic_notes[0]
    else:
        normalized = angle % (2 * np.pi)
        index = int((normalized / (2 * np.pi)) * len(chromatic_notes)) % len(chromatic_notes)
        base_note = chromatic_notes[index]
    return base_note + offset

def volume_from_theta(theta, min_vel=40, max_vel=127):
    # Map theta (0 to pi) to MIDI velocity range (volume)
    theta = np.clip(theta, 0, np.pi)
    vel = int(min_vel + (theta / np.pi) * (max_vel - min_vel))
    return vel

def qubit_state_to_notes_with_volume_constant_duration(alpha, beta, offset=0, step=0, scheme='steady', total_steps=20):
    abs_alpha = np.clip(np.abs(alpha), 0, 1)
    theta = 2 * np.arccos(abs_alpha)
    phi = np.angle(beta) - np.angle(alpha)

    # Map phi to chromatic note pitch with offset per qubit (key shift + octave lowering for Synth)
    pitch_note = chromatic_note_from_angle(phi, offset)
    # Constant duration for all notes
    dur = 1.0  # fixed duration (e.g., 1 quarter note)
    # Map theta to volume (velocity)
    vel = volume_from_theta(theta)

    c = chord.Chord([pitch_note, pitch_note], quarterLength=dur)
    c.volume = volume.Volume(velocity=vel)
    return c

def qubit_to_instrument(index):
    instruments = [
        instrument.Flute(), instrument.Piano(), instrument.Violin(), instrument.Guitar(),
        instrument.Clarinet(), instrument.Trumpet(), instrument.ElectricPiano(),  # Synth substitute, lowered below
        instrument.Oboe(), instrument.Horn(), instrument.Bassoon()
    ]
    return instruments[index % len(instruments)]

def apply_gate(state, gate, qargs, num_qubits):
    temp_circuit = qiskit.QuantumCircuit(num_qubits)
    temp_circuit.append(gate, [temp_circuit.qubits.index(q) for q in qargs])
    return state.evolve(temp_circuit)

def measure_coin(state, coin_qubit):
    probs = state.probabilities_dict()
    keys = list(probs.keys())
    probabilities = np.array([probs[k] for k in keys])
    measured_outcome = np.random.choice(keys, p=probabilities)
    coin_bit = int(measured_outcome[-1 - coin_qubit])
    proj_vec = np.array([1 if k[-1 - coin_qubit] == str(coin_bit) else 0 for k in keys])
    amplitudes = np.array([state.data[int(k, 2)] for k in keys])
    projected_data = amplitudes * proj_vec
    norm = np.linalg.norm(projected_data)
    if norm > 0:
        projected_data /= norm
    new_state_data = np.zeros_like(state.data)
    for i, k in enumerate(keys):
        new_state_data[int(k, 2)] = projected_data[i]
    return Statevector(new_state_data), coin_bit

def circuit_to_music_explicit_with_measure(circuit, coin_qubit=0, total_steps=20):
    num_qubits = circuit.num_qubits
    state = Statevector.from_label('0' * num_qubits)
    qubit_states = {i: [] for i in range(num_qubits)}
    music_score = stream.Score()
    volume_schemes = ['crescendo', 'decrescendo', 'steady', 'random']
    qubit_volume_schemes = [volume_schemes[i % len(volume_schemes)] for i in range(num_qubits)]
    step_counter = 0

    for inst, qargs, _ in circuit.data:
        if inst.name == 'barrier':
            state, _ = measure_coin(state, coin_qubit)
            for i in range(num_qubits):
                reduced = partial_trace(state, [j for j in range(num_qubits) if j != i])
                alpha, beta = reduced.data[0][0], reduced.data[1][0]
                qubit_states[i].append((alpha, beta, step_counter))
            step_counter += 1
        else:
            state = apply_gate(state, inst, qargs, num_qubits)

    for i in range(num_qubits):
        part = stream.Part()
        instr = qubit_to_instrument(i)
        # Lower Synthesizer instrument by 2 octaves (24 semitones)
        if isinstance(instr, instrument.ElectricPiano):
            offset = i * 2 - 24
        else:
            offset = i * 2  # shift keys per qubit by 2 semitones

        part.insert(0, instr)
        scheme = qubit_volume_schemes[i]
        for (alpha, beta, step) in qubit_states[i]:
            note_chord = qubit_state_to_notes_with_volume_constant_duration(alpha, beta, offset, step, scheme, total_steps)
            part.append(note_chord)
        music_score.append(part)

    return music_score, qubit_states

def controlled_increment(qc, control_qubit, position_qubits, ancillas):
    n = len(position_qubits)
    qc.ccx(control_qubit, position_qubits[0], ancillas[0])
    for i in range(1, n-1):
        qc.ccx(ancillas[i-1], position_qubits[i], ancillas[i])
    qc.cx(control_qubit, position_qubits[0])
    for i in range(1, n):
        qc.cx(ancillas[i-1], position_qubits[i])
    for i in reversed(range(1, n-1)):
        qc.ccx(ancillas[i-1], position_qubits[i], ancillas[i])
    qc.ccx(control_qubit, position_qubits[0], ancillas[0])
    qc.barrier()

def controlled_decrement(qc, control_qubit, position_qubits, ancillas):
    for q in position_qubits:
        qc.cx(control_qubit, q)
    controlled_increment(qc, control_qubit, position_qubits, ancillas)
    for q in position_qubits:
        qc.cx(control_qubit, q)
    qc.barrier()

def quantum_walk_step(qc, coin_qubit, position_qubits, ancillas, angle):
    qc.h(coin_qubit)
    qc.barrier()
    controlled_increment(qc, coin_qubit, position_qubits, ancillas)
    qc.x(coin_qubit)
    controlled_decrement(qc, coin_qubit, position_qubits, ancillas)
    qc.x(coin_qubit)
    collision_operator(qc, position_qubits, angle)

def collision_operator(qc, position_qubits, angle):
    for q in position_qubits:
        qc.ry(angle, q)
    qc.barrier()

# Setup quantum circuit parameters
num_position_qubits = 9
num_ancillas = num_position_qubits - 1
num_qubits = 1 + num_position_qubits + num_ancillas  # 1 coin + position + ancillas

qc = qiskit.QuantumCircuit(num_qubits)

coin_qubit = 0
position_qubits = list(range(1, 1 + num_position_qubits))
ancillas = list(range(1 + num_position_qubits, num_qubits))

qc.h(coin_qubit)
qc.barrier()

def random_initialization(qc, position_qubits):
    dim = 2 ** len(position_qubits)
    amplitudes = np.random.rand(dim) + 1j * np.random.rand(dim)
    amplitudes /= np.linalg.norm(amplitudes)
    init_gate = Initialize(amplitudes)
    qc.append(init_gate, position_qubits)
    qc.barrier()

random_initialization(qc, position_qubits)

collision_angles = np.linspace(0, np.pi/2, 20)
for angle in collision_angles:
    quantum_walk_step(qc, coin_qubit, position_qubits, ancillas, angle)

music_score, qubit_states = circuit_to_music_explicit_with_measure(qc, coin_qubit=coin_qubit, total_steps=len(collision_angles))
music_score.show('text')
music_score.write('midi', fp='qw_random_chromatic.mid')
qc.draw(output='mpl', fold=100).savefig('qw_random_chromatic.png')


{0.0} <music21.stream.Part 0x1174d95b0>
    {0.0} <music21.instrument.Flute 'Flute'>
    {0.0} <music21.chord.Chord C4 C4>
    {1.0} <music21.chord.Chord C4 C4>
    {2.0} <music21.chord.Chord C4 C4>
    {3.0} <music21.chord.Chord C4 C4>
    {4.0} <music21.chord.Chord C4 C4>
    {5.0} <music21.chord.Chord C4 C4>
    {6.0} <music21.chord.Chord C4 C4>
    {7.0} <music21.chord.Chord C4 C4>
    {8.0} <music21.chord.Chord C4 C4>
    {9.0} <music21.chord.Chord C4 C4>
    {10.0} <music21.chord.Chord C4 C4>
    {11.0} <music21.chord.Chord C4 C4>
    {12.0} <music21.chord.Chord C4 C4>
    {13.0} <music21.chord.Chord C4 C4>
    {14.0} <music21.chord.Chord C4 C4>
    {15.0} <music21.chord.Chord C4 C4>
    {16.0} <music21.chord.Chord C4 C4>
    {17.0} <music21.chord.Chord C4 C4>
    {18.0} <music21.chord.Chord C4 C4>
    {19.0} <music21.chord.Chord C4 C4>
    {20.0} <music21.chord.Chord C4 C4>
    {21.0} <music21.chord.Chord C4 C4>
    {22.0} <music21.chord.Chord C4 C4>
    {23.0} <music21.chord.Ch

In [16]:
import qiskit
import numpy as np
from qiskit.quantum_info import Statevector, partial_trace
from music21 import stream, chord, instrument, volume
from qiskit.circuit.library import Initialize

# Chromatic scale notes (C4 to B4)
chromatic_notes = list(range(60, 72))  # MIDI notes for one octave chromatic scale

def chromatic_note_from_angle(angle, offset=0):
    if np.isnan(angle) or np.isinf(angle):
        base_note = chromatic_notes[0]
    else:
        normalized = angle % (2 * np.pi)
        index = int((normalized / (2 * np.pi)) * len(chromatic_notes)) % len(chromatic_notes)
        base_note = chromatic_notes[index]
    return base_note + offset

def duration_from_theta(theta, min_duration=0.25, max_duration=2.0):
    theta = np.clip(theta, 0, np.pi)
    dur = min_duration + (theta / np.pi) * (max_duration - min_duration)
    return dur

def volume_scheme(step, scheme, total_steps=20):
    min_vel, max_vel = 40, 127
    if scheme == 'crescendo':
        return int(min_vel + (step / (total_steps-1)) * (max_vel - min_vel))
    elif scheme == 'decrescendo':
        return int(max_vel - (step / (total_steps-1)) * (max_vel - min_vel))
    elif scheme == 'steady':
        return int((min_vel + max_vel) // 2)
    elif scheme == 'random':
        return np.random.randint(min_vel, max_vel+1)
    else:
        return int((min_vel + max_vel) // 2)

def qubit_state_to_notes_with_duration_and_volume(alpha, beta, offset=0, step=0, scheme='steady', total_steps=20):
    abs_alpha = np.clip(np.abs(alpha), 0, 1)
    theta = 2 * np.arccos(abs_alpha)
    phi = np.angle(beta) - np.angle(alpha)

    # Map phi to chromatic note pitch with offset per qubit (key shift)
    pitch_note = chromatic_note_from_angle(phi, offset)
    # Map theta to duration
    dur = duration_from_theta(theta)
    # Map step & scheme to volume velocity
    vel = volume_scheme(step, scheme, total_steps)

    # Create two notes played simultaneously (chord of one note repeated twice)
    # Could be improved to add a second note shifted by some interval if desired
    c = chord.Chord([pitch_note, pitch_note], quarterLength=dur)
    c.volume = volume.Volume(velocity=vel)
    return c

def qubit_to_instrument(index):
    instruments = [
        instrument.Flute(), instrument.Piano(), instrument.Violin(), instrument.Guitar(),
        instrument.Clarinet(), instrument.Trumpet(), instrument.Organ(), instrument.Oboe(),
        instrument.Horn(), instrument.Bassoon()
    ]
    return instruments[index % len(instruments)]

def apply_gate(state, gate, qargs, num_qubits):
    temp_circuit = qiskit.QuantumCircuit(num_qubits)
    temp_circuit.append(gate, [temp_circuit.qubits.index(q) for q in qargs])
    return state.evolve(temp_circuit)

def measure_coin(state, coin_qubit):
    probs = state.probabilities_dict()
    keys = list(probs.keys())
    probabilities = np.array([probs[k] for k in keys])
    measured_outcome = np.random.choice(keys, p=probabilities)
    coin_bit = int(measured_outcome[-1 - coin_qubit])
    proj_vec = np.array([1 if k[-1 - coin_qubit] == str(coin_bit) else 0 for k in keys])
    amplitudes = np.array([state.data[int(k, 2)] for k in keys])
    projected_data = amplitudes * proj_vec
    norm = np.linalg.norm(projected_data)
    if norm > 0:
        projected_data /= norm
    new_state_data = np.zeros_like(state.data)
    for i, k in enumerate(keys):
        new_state_data[int(k, 2)] = projected_data[i]
    return Statevector(new_state_data), coin_bit

def circuit_to_music_explicit_with_measure(circuit, coin_qubit=0, total_steps=20):
    num_qubits = circuit.num_qubits
    state = Statevector.from_label('0' * num_qubits)
    qubit_states = {i: [] for i in range(num_qubits)}
    music_score = stream.Score()
    volume_schemes = ['crescendo', 'decrescendo', 'steady', 'random']
    qubit_volume_schemes = [volume_schemes[i % len(volume_schemes)] for i in range(num_qubits)]
    step_counter = 0

    for inst, qargs, _ in circuit.data:
        if inst.name == 'barrier':
            state, _ = measure_coin(state, coin_qubit)
            for i in range(num_qubits):
                reduced = partial_trace(state, [j for j in range(num_qubits) if j != i])
                alpha, beta = reduced.data[0][0], reduced.data[1][0]
                qubit_states[i].append((alpha, beta, step_counter))
            step_counter += 1
        else:
            state = apply_gate(state, inst, qargs, num_qubits)

    for i in range(num_qubits):
        part = stream.Part()
        part.insert(0, qubit_to_instrument(i))
        offset = i * 2  # shift keys per qubit by 2 semitones
        scheme = qubit_volume_schemes[i]
        for (alpha, beta, step) in qubit_states[i]:
            note_chord = qubit_state_to_notes_with_duration_and_volume(alpha, beta, offset, step, scheme, total_steps)
            part.append(note_chord)
        music_score.append(part)

    return music_score, qubit_states

def controlled_increment(qc, control_qubit, position_qubits, ancillas):
    n = len(position_qubits)
    # first carry
    qc.ccx(control_qubit, position_qubits[0], ancillas[0])
    # propagate carries
    for i in range(1, n-1):
        qc.ccx(ancillas[i-1], position_qubits[i], ancillas[i])
    # flip LSB
    qc.cx(control_qubit, position_qubits[0])
    # flip bits with carry
    for i in range(1, n):
        qc.cx(ancillas[i-1], position_qubits[i])
    # uncompute carries
    for i in reversed(range(1, n-1)):
        qc.ccx(ancillas[i-1], position_qubits[i], ancillas[i])
    qc.ccx(control_qubit, position_qubits[0], ancillas[0])
    qc.barrier()

def controlled_decrement(qc, control_qubit, position_qubits, ancillas):
    # invert all bits controlled by control
    for q in position_qubits:
        qc.cx(control_qubit, q)
    controlled_increment(qc, control_qubit, position_qubits, ancillas)
    for q in position_qubits:
        qc.cx(control_qubit, q)
    qc.barrier()

def quantum_walk_step(qc, coin_qubit, position_qubits, ancillas, angle):
    qc.h(coin_qubit)
    qc.barrier()
    # controlled increment when coin==1
    controlled_increment(qc, coin_qubit, position_qubits, ancillas)
    # controlled decrement when coin==0 (flip coin to control on zero)
    qc.x(coin_qubit)
    controlled_decrement(qc, coin_qubit, position_qubits, ancillas)
    qc.x(coin_qubit)
    collision_operator(qc, position_qubits, angle)

def collision_operator(qc, position_qubits, angle):
    for q in position_qubits:
        qc.ry(angle, q)
    qc.barrier()

# Setup quantum circuit parameters
num_position_qubits = 9
num_ancillas = num_position_qubits - 1
num_qubits = 1 + num_position_qubits + num_ancillas  # 1 coin + position + ancillas

qc = qiskit.QuantumCircuit(num_qubits)

coin_qubit = 0
position_qubits = list(range(1, 1 + num_position_qubits))
ancillas = list(range(1 + num_position_qubits, num_qubits))

qc.h(coin_qubit)
qc.barrier()

def random_initialization(qc, position_qubits):
    dim = 2 ** len(position_qubits)
    amplitudes = np.random.rand(dim) + 1j * np.random.rand(dim)
    amplitudes /= np.linalg.norm(amplitudes)
    init_gate = Initialize(amplitudes)
    qc.append(init_gate, position_qubits)
    qc.barrier()

random_initialization(qc, position_qubits)

collision_angles = np.linspace(0, np.pi/2, 20)
for angle in collision_angles:
    quantum_walk_step(qc, coin_qubit, position_qubits, ancillas, angle)

music_score, qubit_states = circuit_to_music_explicit_with_measure(qc, coin_qubit=coin_qubit, total_steps=len(collision_angles))
music_score.show('text')
music_score.write('midi', fp='qw_random_chr.mid')
qc.draw(output='mpl', fold = 100).savefig('qw_random_chr.png')

{0.0} <music21.stream.Part 0x119f1eea0>
    {0.0} <music21.instrument.Flute 'Flute'>
    {0.0} <music21.chord.Chord C4 C4>
    {0.25} <music21.chord.Chord C4 C4>
    {0.5} <music21.chord.Chord C4 C4>
    {2.5} <music21.chord.Chord C4 C4>
    {4.5} <music21.chord.Chord C4 C4>
    {4.75} <music21.chord.Chord C4 C4>
    {5.0} <music21.chord.Chord C4 C4>
    {7.0} <music21.chord.Chord C4 C4>
    {7.25} <music21.chord.Chord C4 C4>
    {7.5} <music21.chord.Chord C4 C4>
    {9.5} <music21.chord.Chord C4 C4>
    {11.5} <music21.chord.Chord C4 C4>
    {11.75} <music21.chord.Chord C4 C4>
    {13.75} <music21.chord.Chord C4 C4>
    {15.75} <music21.chord.Chord C4 C4>
    {16.0} <music21.chord.Chord C4 C4>
    {16.25} <music21.chord.Chord C4 C4>
    {18.25} <music21.chord.Chord C4 C4>
    {18.5} <music21.chord.Chord C4 C4>
    {18.75} <music21.chord.Chord C4 C4>
    {20.75} <music21.chord.Chord C4 C4>
    {22.75} <music21.chord.Chord C4 C4>
    {23.0} <music21.chord.Chord C4 C4>
    {23.25} <music

In [17]:
import qiskit
import numpy as np
from qiskit.quantum_info import Statevector, partial_trace
from music21 import stream, chord, instrument, volume
from qiskit.circuit.library import Initialize

# Chromatic scale notes (C4 to B4)
chromatic_notes = list(range(60, 72))  # MIDI notes for one octave chromatic scale

def chromatic_note_from_angle(angle, offset=0):
    if np.isnan(angle) or np.isinf(angle):
        base_note = chromatic_notes[0]
    else:
        normalized = angle % (2 * np.pi)
        index = int((normalized / (2 * np.pi)) * len(chromatic_notes)) % len(chromatic_notes)
        base_note = chromatic_notes[index]
    return base_note + offset

def duration_from_theta(theta, min_duration=0.25, max_duration=2.0):
    theta = np.clip(theta, 0, np.pi)
    dur = min_duration + (theta / np.pi) * (max_duration - min_duration)
    return dur

def volume_scheme(step, scheme, total_steps=20):
    min_vel, max_vel = 40, 127
    if scheme == 'crescendo':
        return int(min_vel + (step / (total_steps - 1)) * (max_vel - min_vel))
    elif scheme == 'decrescendo':
        return int(max_vel - (step / (total_steps - 1)) * (max_vel - min_vel))
    elif scheme == 'steady':
        return int((min_vel + max_vel) // 2)
    elif scheme == 'random':
        return np.random.randint(min_vel, max_vel + 1)
    else:
        return int((min_vel + max_vel) // 2)

def qubit_state_to_notes_with_duration_and_volume(alpha, beta, offset=0, step=0, scheme='steady', total_steps=20):
    abs_alpha = np.clip(np.abs(alpha), 0, 1)
    theta = 2 * np.arccos(abs_alpha)
    phi = np.angle(beta) - np.angle(alpha)

    pitch_note = chromatic_note_from_angle(phi, offset)
    dur = duration_from_theta(theta)
    vel = volume_scheme(step, scheme, total_steps)

    c = chord.Chord([pitch_note, pitch_note], quarterLength=dur)
    c.volume = volume.Volume(velocity=vel)
    return c

def qubit_to_instrument(index):
    instruments = [
        instrument.Flute(), instrument.Piano(), instrument.Violin(), instrument.Guitar(),
        instrument.Clarinet(), instrument.Trumpet(), instrument.ElectricPiano(), instrument.Oboe(),
        instrument.Horn(), instrument.Bassoon()
    ]
    return instruments[index % len(instruments)]

def apply_gate(state, gate, qargs, num_qubits):
    temp_circuit = qiskit.QuantumCircuit(num_qubits)
    temp_circuit.append(gate, [temp_circuit.qubits.index(q) for q in qargs])
    return state.evolve(temp_circuit)

def measure_coin(state, coin_qubit):
    probs = state.probabilities_dict()
    keys = list(probs.keys())
    probabilities = np.array([probs[k] for k in keys])
    measured_outcome = np.random.choice(keys, p=probabilities)
    coin_bit = int(measured_outcome[-1 - coin_qubit])
    proj_vec = np.array([1 if k[-1 - coin_qubit] == str(coin_bit) else 0 for k in keys])
    amplitudes = np.array([state.data[int(k, 2)] for k in keys])
    projected_data = amplitudes * proj_vec
    norm = np.linalg.norm(projected_data)
    if norm > 0:
        projected_data /= norm
    new_state_data = np.zeros_like(state.data)
    for i, k in enumerate(keys):
        new_state_data[int(k, 2)] = projected_data[i]
    return Statevector(new_state_data), coin_bit

def circuit_to_music_explicit_with_measure(circuit, coin_qubit=0, total_steps=20):
    num_qubits = circuit.num_qubits
    state = Statevector.from_label('0' * num_qubits)
    qubit_states = {i: [] for i in range(num_qubits)}
    music_score = stream.Score()
    volume_schemes = ['crescendo', 'decrescendo', 'steady', 'random']
    qubit_volume_schemes = [volume_schemes[i % len(volume_schemes)] for i in range(num_qubits)]
    step_counter = 0

    for inst, qargs, _ in circuit.data:
        if inst.name == 'barrier':
            state, _ = measure_coin(state, coin_qubit)
            for i in range(num_qubits):
                reduced = partial_trace(state, [j for j in range(num_qubits) if j != i])
                alpha, beta = reduced.data[0][0], reduced.data[1][0]
                qubit_states[i].append((alpha, beta, step_counter))
            step_counter += 1
        else:
            state = apply_gate(state, inst, qargs, num_qubits)

    for i in range(num_qubits):
        part = stream.Part()
        part.insert(0, qubit_to_instrument(i))
        offset = i * 2 - 36  # lowered by 3 octaves total
        scheme = qubit_volume_schemes[i]
        for (alpha, beta, step) in qubit_states[i]:
            note_chord = qubit_state_to_notes_with_duration_and_volume(alpha, beta, offset, step, scheme, total_steps)
            part.append(note_chord)
        music_score.append(part)

    return music_score, qubit_states

def controlled_increment(qc, control_qubit, position_qubits, ancillas):
    n = len(position_qubits)
    qc.ccx(control_qubit, position_qubits[0], ancillas[0])
    for i in range(1, n - 1):
        qc.ccx(ancillas[i - 1], position_qubits[i], ancillas[i])
    qc.cx(control_qubit, position_qubits[0])
    for i in range(1, n):
        qc.cx(ancillas[i - 1], position_qubits[i])
    for i in reversed(range(1, n - 1)):
        qc.ccx(ancillas[i - 1], position_qubits[i], ancillas[i])
    qc.ccx(control_qubit, position_qubits[0], ancillas[0])
    qc.barrier()

def controlled_decrement(qc, control_qubit, position_qubits, ancillas):
    for q in position_qubits:
        qc.cx(control_qubit, q)
    controlled_increment(qc, control_qubit, position_qubits, ancillas)
    for q in position_qubits:
        qc.cx(control_qubit, q)
    qc.barrier()

def quantum_walk_step(qc, coin_qubit, position_qubits, ancillas, angle):
    qc.h(coin_qubit)
    qc.barrier()
    controlled_increment(qc, coin_qubit, position_qubits, ancillas)
    qc.x(coin_qubit)
    controlled_decrement(qc, coin_qubit, position_qubits, ancillas)
    qc.x(coin_qubit)
    collision_operator(qc, position_qubits, angle)

def collision_operator(qc, position_qubits, angle):
    for q in position_qubits:
        qc.ry(angle, q)
    qc.barrier()

# Setup quantum circuit parameters
num_position_qubits = 9
num_ancillas = num_position_qubits - 1
num_qubits = 1 + num_position_qubits + num_ancillas  # 1 coin + position + ancillas

qc = qiskit.QuantumCircuit(num_qubits)
coin_qubit = 0
position_qubits = list(range(1, 1 + num_position_qubits))
ancillas = list(range(1 + num_position_qubits, num_qubits))

qc.h(coin_qubit)
qc.barrier()

def random_initialization(qc, position_qubits):
    dim = 2 ** len(position_qubits)
    amplitudes = np.random.rand(dim) + 1j * np.random.rand(dim)
    amplitudes /= np.linalg.norm(amplitudes)
    init_gate = Initialize(amplitudes)
    qc.append(init_gate, position_qubits)
    qc.barrier()

random_initialization(qc, position_qubits)

collision_angles = np.linspace(0, np.pi / 2, 20)
for angle in collision_angles:
    quantum_walk_step(qc, coin_qubit, position_qubits, ancillas, angle)

music_score, qubit_states = circuit_to_music_explicit_with_measure(qc, coin_qubit=coin_qubit, total_steps=len(collision_angles))
music_score.show('text')
music_score.write('midi', fp='qw_random_chr_lower.mid')
qc.draw(output='mpl', fold=100).savefig('qw_random_chr_lower.png')


{0.0} <music21.stream.Part 0x11b2d6ea0>
    {0.0} <music21.instrument.Flute 'Flute'>
    {0.0} <music21.chord.Chord C1 C1>
    {2.0} <music21.chord.Chord C1 C1>
    {4.0} <music21.chord.Chord C1 C1>
    {6.0} <music21.chord.Chord C1 C1>
    {8.0} <music21.chord.Chord C1 C1>
    {8.25} <music21.chord.Chord C1 C1>
    {8.5} <music21.chord.Chord C1 C1>
    {10.5} <music21.chord.Chord C1 C1>
    {10.75} <music21.chord.Chord C1 C1>
    {11.0} <music21.chord.Chord C1 C1>
    {13.0} <music21.chord.Chord C1 C1>
    {15.0} <music21.chord.Chord C1 C1>
    {15.25} <music21.chord.Chord C1 C1>
    {15.5} <music21.chord.Chord C1 C1>
    {15.75} <music21.chord.Chord C1 C1>
    {17.75} <music21.chord.Chord C1 C1>
    {19.75} <music21.chord.Chord C1 C1>
    {20.0} <music21.chord.Chord C1 C1>
    {20.25} <music21.chord.Chord C1 C1>
    {20.5} <music21.chord.Chord C1 C1>
    {22.5} <music21.chord.Chord C1 C1>
    {24.5} <music21.chord.Chord C1 C1>
    {24.75} <music21.chord.Chord C1 C1>
    {25.0} <music

In [20]:
import qiskit
import numpy as np
from qiskit.quantum_info import Statevector, partial_trace
from music21 import stream, chord, instrument, volume
from qiskit.circuit.library import Initialize

# Pentatonic scale notes (C major pentatonic over one octave)
pentatonic_notes = [60, 62, 64, 67, 69]  # C, D, E, G, A in MIDI

def chromatic_note_from_angle(angle, offset=0):
    if np.isnan(angle) or np.isinf(angle):
        base_note = pentatonic_notes[0]
    else:
        normalized = angle % (2 * np.pi)
        index = int((normalized / (2 * np.pi)) * len(pentatonic_notes)) % len(pentatonic_notes)
        base_note = pentatonic_notes[index]
    return base_note + offset

def duration_from_theta(theta, min_duration=0.25, max_duration=2.0):
    theta = np.clip(theta, 0, np.pi)
    dur = min_duration + (theta / np.pi) * (max_duration - min_duration)
    return dur

def volume_scheme(step, scheme, total_steps=20):
    min_vel, max_vel = 40, 127
    if scheme == 'crescendo':
        return int(min_vel + (step / (total_steps - 1)) * (max_vel - min_vel))
    elif scheme == 'decrescendo':
        return int(max_vel - (step / (total_steps - 1)) * (max_vel - min_vel))
    elif scheme == 'steady':
        return int((min_vel + max_vel) // 2)
    elif scheme == 'random':
        return np.random.randint(min_vel, max_vel + 1)
    else:
        return int((min_vel + max_vel) // 2)

def qubit_state_to_notes_with_duration_and_volume(alpha, beta, offset=0, step=0, scheme='steady', total_steps=20):
    abs_alpha = np.clip(np.abs(alpha), 0, 1)
    theta = 2 * np.arccos(abs_alpha)
    phi = np.angle(beta) - np.angle(alpha)

    pitch_note = chromatic_note_from_angle(phi, offset)
    dur = duration_from_theta(theta)
    vel = volume_scheme(step, scheme, total_steps)

    c = chord.Chord([pitch_note, pitch_note], quarterLength=dur)
    c.volume = volume.Volume(velocity=vel)
    return c

def qubit_to_instrument(index):
    instruments = [
        instrument.Flute(), instrument.Piano(), instrument.Violin(), instrument.Guitar(),
        instrument.Clarinet(), instrument.Trumpet(), instrument.ElectricPiano(), instrument.Oboe(),
        instrument.Horn(), instrument.Bassoon()
    ]
    return instruments[index % len(instruments)]

def apply_gate(state, gate, qargs, num_qubits):
    temp_circuit = qiskit.QuantumCircuit(num_qubits)
    temp_circuit.append(gate, [temp_circuit.qubits.index(q) for q in qargs])
    return state.evolve(temp_circuit)

def measure_coin(state, coin_qubit):
    probs = state.probabilities_dict()
    keys = list(probs.keys())
    probabilities = np.array([probs[k] for k in keys])
    measured_outcome = np.random.choice(keys, p=probabilities)
    coin_bit = int(measured_outcome[-1 - coin_qubit])
    proj_vec = np.array([1 if k[-1 - coin_qubit] == str(coin_bit) else 0 for k in keys])
    amplitudes = np.array([state.data[int(k, 2)] for k in keys])
    projected_data = amplitudes * proj_vec
    norm = np.linalg.norm(projected_data)
    if norm > 0:
        projected_data /= norm
    new_state_data = np.zeros_like(state.data)
    for i, k in enumerate(keys):
        new_state_data[int(k, 2)] = projected_data[i]
    return Statevector(new_state_data), coin_bit

def circuit_to_music_explicit_with_measure(circuit, coin_qubit=0, total_steps=20):
    num_qubits = circuit.num_qubits
    state = Statevector.from_label('0' * num_qubits)
    qubit_states = {i: [] for i in range(num_qubits)}
    music_score = stream.Score()
    volume_schemes = ['crescendo', 'decrescendo', 'steady', 'random']
    qubit_volume_schemes = [volume_schemes[i % len(volume_schemes)] for i in range(num_qubits)]
    step_counter = 0

    for inst, qargs, _ in circuit.data:
        if inst.name == 'barrier':
            state, _ = measure_coin(state, coin_qubit)
            for i in range(num_qubits):
                reduced = partial_trace(state, [j for j in range(num_qubits) if j != i])
                alpha, beta = reduced.data[0][0], reduced.data[1][0]
                qubit_states[i].append((alpha, beta, step_counter))
            step_counter += 1
        else:
            state = apply_gate(state, inst, qargs, num_qubits)

    for i in range(num_qubits):
        part = stream.Part()
        part.insert(0, qubit_to_instrument(i))
        offset = i * 2 - 36  # Lowered 3 octaves as before
        scheme = qubit_volume_schemes[i]
        for (alpha, beta, step) in qubit_states[i]:
            note_chord = qubit_state_to_notes_with_duration_and_volume(alpha, beta, offset, step, scheme, total_steps)
            part.append(note_chord)
        music_score.append(part)

    return music_score, qubit_states

def controlled_increment(qc, control_qubit, position_qubits, ancillas):
    n = len(position_qubits)
    qc.ccx(control_qubit, position_qubits[0], ancillas[0])
    for i in range(1, n - 1):
        qc.ccx(ancillas[i - 1], position_qubits[i], ancillas[i])
    qc.cx(control_qubit, position_qubits[0])
    for i in range(1, n):
        qc.cx(ancillas[i - 1], position_qubits[i])
    for i in reversed(range(1, n - 1)):
        qc.ccx(ancillas[i - 1], position_qubits[i], ancillas[i])
    qc.ccx(control_qubit, position_qubits[0], ancillas[0])
    qc.barrier()

def controlled_decrement(qc, control_qubit, position_qubits, ancillas):
    for q in position_qubits:
        qc.cx(control_qubit, q)
    controlled_increment(qc, control_qubit, position_qubits, ancillas)
    for q in position_qubits:
        qc.cx(control_qubit, q)
    qc.barrier()

def quantum_walk_step(qc, coin_qubit, position_qubits, ancillas, angle):
    qc.h(coin_qubit)
    qc.barrier()
    controlled_increment(qc, coin_qubit, position_qubits, ancillas)
    qc.x(coin_qubit)
    controlled_decrement(qc, coin_qubit, position_qubits, ancillas)
    qc.x(coin_qubit)
    collision_operator(qc, position_qubits, angle)

def collision_operator(qc, position_qubits, angle):
    for q in position_qubits:
        qc.ry(angle, q)
    qc.barrier()

# Setup quantum circuit parameters
num_position_qubits = 9
num_ancillas = num_position_qubits - 1
num_qubits = 1 + num_position_qubits + num_ancillas  # 1 coin + position + ancillas

qc = qiskit.QuantumCircuit(num_qubits)
coin_qubit = 0
position_qubits = list(range(1, 1 + num_position_qubits))
ancillas = list(range(1 + num_position_qubits, num_qubits))

qc.h(coin_qubit)
qc.barrier()

def random_initialization(qc, position_qubits):
    dim = 2 ** len(position_qubits)
    amplitudes = np.random.rand(dim) + 1j * np.random.rand(dim)
    amplitudes /= np.linalg.norm(amplitudes)
    init_gate = Initialize(amplitudes)
    qc.append(init_gate, position_qubits)
    qc.barrier()

random_initialization(qc, position_qubits)

collision_angles = np.linspace(0, np.pi / 2, 20)
for angle in collision_angles:
    quantum_walk_step(qc, coin_qubit, position_qubits, ancillas, angle)

music_score, qubit_states = circuit_to_music_explicit_with_measure(qc, coin_qubit=coin_qubit, total_steps=len(collision_angles))
music_score.show('text')
music_score.write('midi', fp='qw_random_penta.mid')  # Changed filename
qc.draw(output='mpl', fold=100).savefig('qw_random_penta.png')


{0.0} <music21.stream.Part 0x1155b3080>
    {0.0} <music21.instrument.Flute 'Flute'>
    {0.0} <music21.chord.Chord C1 C1>
    {2.0} <music21.chord.Chord C1 C1>
    {4.0} <music21.chord.Chord C1 C1>
    {4.25} <music21.chord.Chord C1 C1>
    {4.5} <music21.chord.Chord C1 C1>
    {6.5} <music21.chord.Chord C1 C1>
    {8.5} <music21.chord.Chord C1 C1>
    {8.75} <music21.chord.Chord C1 C1>
    {10.75} <music21.chord.Chord C1 C1>
    {12.75} <music21.chord.Chord C1 C1>
    {13.0} <music21.chord.Chord C1 C1>
    {13.25} <music21.chord.Chord C1 C1>
    {15.25} <music21.chord.Chord C1 C1>
    {15.5} <music21.chord.Chord C1 C1>
    {15.75} <music21.chord.Chord C1 C1>
    {17.75} <music21.chord.Chord C1 C1>
    {19.75} <music21.chord.Chord C1 C1>
    {20.0} <music21.chord.Chord C1 C1>
    {20.25} <music21.chord.Chord C1 C1>
    {20.5} <music21.chord.Chord C1 C1>
    {22.5} <music21.chord.Chord C1 C1>
    {24.5} <music21.chord.Chord C1 C1>
    {24.75} <music21.chord.Chord C1 C1>
    {26.75} <mu

In [21]:
import qiskit
import numpy as np
from qiskit.quantum_info import Statevector, partial_trace
from music21 import stream, chord, instrument, volume
from qiskit.circuit.library import Initialize

# Pentatonic scale notes (C major pentatonic over one octave)
pentatonic_notes = [60, 62, 64, 67, 69]  # C, D, E, G, A in MIDI

def chromatic_note_from_angle(angle, offset=0):
    if np.isnan(angle) or np.isinf(angle):
        base_note = pentatonic_notes[0]
    else:
        normalized = angle % (2 * np.pi)
        index = int((normalized / (2 * np.pi)) * len(pentatonic_notes)) % len(pentatonic_notes)
        base_note = pentatonic_notes[index]
    return base_note + offset

def duration_from_theta(theta, min_duration=0.25, max_duration=2.0):
    theta = np.clip(theta, 0, np.pi)
    dur = min_duration + (theta / np.pi) * (max_duration - min_duration)
    return dur

def volume_scheme(step, scheme, total_steps=20):
    min_vel, max_vel = 40, 127
    if scheme == 'crescendo':
        return int(min_vel + (step / (total_steps - 1)) * (max_vel - min_vel))
    elif scheme == 'decrescendo':
        return int(max_vel - (step / (total_steps - 1)) * (max_vel - min_vel))
    elif scheme == 'steady':
        return int((min_vel + max_vel) // 2)
    elif scheme == 'random':
        return np.random.randint(min_vel, max_vel + 1)
    else:
        return int((min_vel + max_vel) // 2)

def qubit_state_to_notes_with_duration_and_volume(alpha, beta, offset=0, step=0, scheme='steady', total_steps=20):
    abs_alpha = np.clip(np.abs(alpha), 0, 1)
    theta = 2 * np.arccos(abs_alpha)
    phi = np.angle(beta) - np.angle(alpha)

    pitch_note = chromatic_note_from_angle(phi, offset)
    dur = duration_from_theta(theta)
    vel = volume_scheme(step, scheme, total_steps)

    c = chord.Chord([pitch_note, pitch_note], quarterLength=dur)
    c.volume = volume.Volume(velocity=vel)
    return c

def qubit_to_instrument(index):
    instruments = [
        instrument.Flute(), instrument.Piano(), instrument.Violin(), instrument.Guitar(),
        instrument.Clarinet(), instrument.Trumpet(), instrument.ElectricPiano(), instrument.Oboe(),
        instrument.Horn(), instrument.Bassoon()
    ]
    return instruments[index % len(instruments)]

def apply_gate(state, gate, qargs, num_qubits):
    temp_circuit = qiskit.QuantumCircuit(num_qubits)
    temp_circuit.append(gate, [temp_circuit.qubits.index(q) for q in qargs])
    return state.evolve(temp_circuit)

def measure_coin(state, coin_qubit):
    probs = state.probabilities_dict()
    keys = list(probs.keys())
    probabilities = np.array([probs[k] for k in keys])
    measured_outcome = np.random.choice(keys, p=probabilities)
    coin_bit = int(measured_outcome[-1 - coin_qubit])
    proj_vec = np.array([1 if k[-1 - coin_qubit] == str(coin_bit) else 0 for k in keys])
    amplitudes = np.array([state.data[int(k, 2)] for k in keys])
    projected_data = amplitudes * proj_vec
    norm = np.linalg.norm(projected_data)
    if norm > 0:
        projected_data /= norm
    new_state_data = np.zeros_like(state.data)
    for i, k in enumerate(keys):
        new_state_data[int(k, 2)] = projected_data[i]
    return Statevector(new_state_data), coin_bit

def circuit_to_music_explicit_with_measure(circuit, coin_qubit=0, total_steps=20):
    num_qubits = circuit.num_qubits
    state = Statevector.from_label('0' * num_qubits)
    qubit_states = {i: [] for i in range(num_qubits)}
    music_score = stream.Score()
    volume_schemes = ['crescendo', 'decrescendo', 'steady', 'random']
    qubit_volume_schemes = [volume_schemes[i % len(volume_schemes)] for i in range(num_qubits)]
    step_counter = 0

    for inst, qargs, _ in circuit.data:
        if inst.name == 'barrier':
            state, _ = measure_coin(state, coin_qubit)
            for i in range(num_qubits):
                reduced = partial_trace(state, [j for j in range(num_qubits) if j != i])
                alpha, beta = reduced.data[0][0], reduced.data[1][0]
                qubit_states[i].append((alpha, beta, step_counter))
            step_counter += 1
        else:
            state = apply_gate(state, inst, qargs, num_qubits)

    for i in range(num_qubits):
        part = stream.Part()
        part.insert(0, qubit_to_instrument(i))
        offset = i * 2 - 36  # Lowered 3 octaves as before
        scheme = qubit_volume_schemes[i]
        for (alpha, beta, step) in qubit_states[i]:
            note_chord = qubit_state_to_notes_with_duration_and_volume(alpha, beta, offset, step, scheme, total_steps)
            part.append(note_chord)
        music_score.append(part)

    return music_score, qubit_states

def controlled_increment(qc, control_qubit, position_qubits, ancillas):
    n = len(position_qubits)
    qc.ccx(control_qubit, position_qubits[0], ancillas[0])
    for i in range(1, n - 1):
        qc.ccx(ancillas[i - 1], position_qubits[i], ancillas[i])
    qc.cx(control_qubit, position_qubits[0])
    for i in range(1, n):
        qc.cx(ancillas[i - 1], position_qubits[i])
    for i in reversed(range(1, n - 1)):
        qc.ccx(ancillas[i - 1], position_qubits[i], ancillas[i])
    qc.ccx(control_qubit, position_qubits[0], ancillas[0])
    qc.barrier()

def controlled_decrement(qc, control_qubit, position_qubits, ancillas):
    for q in position_qubits:
        qc.cx(control_qubit, q)
    controlled_increment(qc, control_qubit, position_qubits, ancillas)
    for q in position_qubits:
        qc.cx(control_qubit, q)
    qc.barrier()

def quantum_walk_step(qc, coin_qubit, position_qubits, ancillas, angle):
    qc.h(coin_qubit)
    qc.barrier()
    controlled_increment(qc, coin_qubit, position_qubits, ancillas)
    qc.x(coin_qubit)
    controlled_decrement(qc, coin_qubit, position_qubits, ancillas)
    qc.x(coin_qubit)
    collision_operator(qc, position_qubits, angle)

def collision_operator(qc, position_qubits, angle):
    for q in position_qubits:
        qc.ry(angle, q)
    qc.barrier()

# Setup quantum circuit parameters
num_position_qubits = 9
num_ancillas = num_position_qubits - 1
num_qubits = 1 + num_position_qubits + num_ancillas  # 1 coin + position + ancillas

qc = qiskit.QuantumCircuit(num_qubits)
coin_qubit = 0
position_qubits = list(range(1, 1 + num_position_qubits))
ancillas = list(range(1 + num_position_qubits, num_qubits))

qc.h(coin_qubit)
qc.barrier()

def gaussian_initialization(qc, position_qubits, center=None, sigma=10):
    n = len(position_qubits)
    dim = 2 ** n
    if center is None:
        center = dim // 2
    x = np.arange(dim)
    amplitudes = np.exp(-0.5 * ((x - center) / sigma) ** 2)
    amplitudes /= np.linalg.norm(amplitudes)
    init_gate = Initialize(amplitudes)
    qc.append(init_gate, position_qubits)
    qc.barrier()

gaussian_initialization(qc, position_qubits)

collision_angles = np.linspace(0, np.pi / 2, 20)
for angle in collision_angles:
    quantum_walk_step(qc, coin_qubit, position_qubits, ancillas, angle)

music_score, qubit_states = circuit_to_music_explicit_with_measure(qc, coin_qubit=coin_qubit, total_steps=len(collision_angles))
music_score.show('text')
music_score.write('midi', fp='qw_gaussian_penta.mid')  # Changed filename
qc.draw(output='mpl', fold=100).savefig('qw_gaussian_penta.png')


{0.0} <music21.stream.Part 0x11af280e0>
    {0.0} <music21.instrument.Flute 'Flute'>
    {0.0} <music21.chord.Chord C1 C1>
    {0.25} <music21.chord.Chord C1 C1>
    {0.5} <music21.chord.Chord C1 C1>
    {0.75} <music21.chord.Chord C1 C1>
    {1.0} <music21.chord.Chord C1 C1>
    {3.0} <music21.chord.Chord C1 C1>
    {5.0} <music21.chord.Chord C1 C1>
    {5.25} <music21.chord.Chord C1 C1>
    {5.5} <music21.chord.Chord C1 C1>
    {5.75} <music21.chord.Chord C1 C1>
    {7.75} <music21.chord.Chord C1 C1>
    {9.75} <music21.chord.Chord C1 C1>
    {10.0} <music21.chord.Chord C1 C1>
    {10.25} <music21.chord.Chord C1 C1>
    {10.5} <music21.chord.Chord C1 C1>
    {12.5} <music21.chord.Chord C1 C1>
    {14.5} <music21.chord.Chord C1 C1>
    {14.75} <music21.chord.Chord C1 C1>
    {15.0} <music21.chord.Chord C1 C1>
    {15.25} <music21.chord.Chord C1 C1>
    {17.25} <music21.chord.Chord C1 C1>
    {19.25} <music21.chord.Chord C1 C1>
    {19.5} <music21.chord.Chord C1 C1>
    {19.75} <music2

In [22]:
import qiskit
import numpy as np
from qiskit.quantum_info import Statevector, partial_trace
from music21 import stream, chord, instrument, volume
from qiskit.circuit.library import Initialize

# Chromatic scale notes (C4 to B4)
chromatic_notes = list(range(60, 72))  # MIDI notes for one octave chromatic scale

def chromatic_note_from_angle(angle, offset=0):
    if np.isnan(angle) or np.isinf(angle):
        base_note = chromatic_notes[0]
    else:
        normalized = angle % (2 * np.pi)
        index = int((normalized / (2 * np.pi)) * len(chromatic_notes)) % len(chromatic_notes)
        base_note = chromatic_notes[index]
    return base_note + offset

def duration_from_theta(theta, min_duration=0.25, max_duration=2.0):
    theta = np.clip(theta, 0, np.pi)
    dur = min_duration + (theta / np.pi) * (max_duration - min_duration)
    return dur

def volume_scheme(step, scheme, total_steps=20):
    min_vel, max_vel = 40, 127
    if scheme == 'crescendo':
        return int(min_vel + (step / (total_steps - 1)) * (max_vel - min_vel))
    elif scheme == 'decrescendo':
        return int(max_vel - (step / (total_steps - 1)) * (max_vel - min_vel))
    elif scheme == 'steady':
        return int((min_vel + max_vel) // 2)
    elif scheme == 'random':
        return np.random.randint(min_vel, max_vel + 1)
    else:
        return int((min_vel + max_vel) // 2)

def qubit_state_to_notes_with_duration_and_volume(alpha, beta, offset=0, step=0, scheme='steady', total_steps=20):
    abs_alpha = np.clip(np.abs(alpha), 0, 1)
    theta = 2 * np.arccos(abs_alpha)
    phi = np.angle(beta) - np.angle(alpha)

    pitch_note = chromatic_note_from_angle(phi, offset)
    dur = duration_from_theta(theta)
    vel = volume_scheme(step, scheme, total_steps)

    c = chord.Chord([pitch_note, pitch_note], quarterLength=dur)
    c.volume = volume.Volume(velocity=vel)
    return c

def qubit_to_instrument(index):
    instruments = [
        instrument.Flute(), instrument.Piano(), instrument.Violin(), instrument.Guitar(),
        instrument.Clarinet(), instrument.Trumpet(), instrument.ElectricPiano(), instrument.Oboe(),
        instrument.Horn(), instrument.Bassoon()
    ]
    return instruments[index % len(instruments)]

def apply_gate(state, gate, qargs, num_qubits):
    temp_circuit = qiskit.QuantumCircuit(num_qubits)
    temp_circuit.append(gate, [temp_circuit.qubits.index(q) for q in qargs])
    return state.evolve(temp_circuit)

def measure_coin(state, coin_qubit):
    probs = state.probabilities_dict()
    keys = list(probs.keys())
    probabilities = np.array([probs[k] for k in keys])
    measured_outcome = np.random.choice(keys, p=probabilities)
    coin_bit = int(measured_outcome[-1 - coin_qubit])
    proj_vec = np.array([1 if k[-1 - coin_qubit] == str(coin_bit) else 0 for k in keys])
    amplitudes = np.array([state.data[int(k, 2)] for k in keys])
    projected_data = amplitudes * proj_vec
    norm = np.linalg.norm(projected_data)
    if norm > 0:
        projected_data /= norm
    new_state_data = np.zeros_like(state.data)
    for i, k in enumerate(keys):
        new_state_data[int(k, 2)] = projected_data[i]
    return Statevector(new_state_data), coin_bit

def circuit_to_music_explicit_with_measure(circuit, coin_qubit=0, total_steps=20):
    num_qubits = circuit.num_qubits
    state = Statevector.from_label('0' * num_qubits)
    qubit_states = {i: [] for i in range(num_qubits)}
    music_score = stream.Score()
    volume_schemes = ['crescendo', 'decrescendo', 'steady', 'random']
    qubit_volume_schemes = [volume_schemes[i % len(volume_schemes)] for i in range(num_qubits)]
    step_counter = 0

    for inst, qargs, _ in circuit.data:
        if inst.name == 'barrier':
            state, _ = measure_coin(state, coin_qubit)
            for i in range(num_qubits):
                reduced = partial_trace(state, [j for j in range(num_qubits) if j != i])
                alpha, beta = reduced.data[0][0], reduced.data[1][0]
                qubit_states[i].append((alpha, beta, step_counter))
            step_counter += 1
        else:
            state = apply_gate(state, inst, qargs, num_qubits)

    for i in range(num_qubits):
        part = stream.Part()
        part.insert(0, qubit_to_instrument(i))
        offset = i * 2 - 36  # lowered by 3 octaves total
        scheme = qubit_volume_schemes[i]
        for (alpha, beta, step) in qubit_states[i]:
            note_chord = qubit_state_to_notes_with_duration_and_volume(alpha, beta, offset, step, scheme, total_steps)
            part.append(note_chord)
        music_score.append(part)

    return music_score, qubit_states

def controlled_increment(qc, control_qubit, position_qubits, ancillas):
    n = len(position_qubits)
    qc.ccx(control_qubit, position_qubits[0], ancillas[0])
    for i in range(1, n - 1):
        qc.ccx(ancillas[i - 1], position_qubits[i], ancillas[i])
    qc.cx(control_qubit, position_qubits[0])
    for i in range(1, n):
        qc.cx(ancillas[i - 1], position_qubits[i])
    for i in reversed(range(1, n - 1)):
        qc.ccx(ancillas[i - 1], position_qubits[i], ancillas[i])
    qc.ccx(control_qubit, position_qubits[0], ancillas[0])
    qc.barrier()

def controlled_decrement(qc, control_qubit, position_qubits, ancillas):
    for q in position_qubits:
        qc.cx(control_qubit, q)
    controlled_increment(qc, control_qubit, position_qubits, ancillas)
    for q in position_qubits:
        qc.cx(control_qubit, q)
    qc.barrier()

def quantum_walk_step(qc, coin_qubit, position_qubits, ancillas, angle):
    qc.h(coin_qubit)
    qc.barrier()
    controlled_increment(qc, coin_qubit, position_qubits, ancillas)
    qc.x(coin_qubit)
    controlled_decrement(qc, coin_qubit, position_qubits, ancillas)
    qc.x(coin_qubit)
    collision_operator(qc, position_qubits, angle)

def collision_operator(qc, position_qubits, angle):
    for q in position_qubits:
        qc.ry(angle, q)
    qc.barrier()

# Setup quantum circuit parameters
num_position_qubits = 9
num_ancillas = num_position_qubits - 1
num_qubits = 1 + num_position_qubits + num_ancillas  # 1 coin + position + ancillas

qc = qiskit.QuantumCircuit(num_qubits)
coin_qubit = 0
position_qubits = list(range(1, 1 + num_position_qubits))
ancillas = list(range(1 + num_position_qubits, num_qubits))

qc.h(coin_qubit)
qc.barrier()



gaussian_initialization(qc, position_qubits)

collision_angles = np.linspace(0, np.pi / 2, 20)
for angle in collision_angles:
    quantum_walk_step(qc, coin_qubit, position_qubits, ancillas, angle)

music_score, qubit_states = circuit_to_music_explicit_with_measure(qc, coin_qubit=coin_qubit, total_steps=len(collision_angles))
music_score.show('text')
music_score.write('midi', fp='qw_gaussian_chr_lower.mid')
qc.draw(output='mpl', fold=100).savefig('qw_gaussian_chr_lower.png')


{0.0} <music21.stream.Part 0x118272270>
    {0.0} <music21.instrument.Flute 'Flute'>
    {0.0} <music21.chord.Chord C1 C1>
    {2.0} <music21.chord.Chord C1 C1>
    {4.0} <music21.chord.Chord C1 C1>
    {4.25} <music21.chord.Chord C1 C1>
    {4.5} <music21.chord.Chord C1 C1>
    {6.5} <music21.chord.Chord C1 C1>
    {8.5} <music21.chord.Chord C1 C1>
    {8.75} <music21.chord.Chord C1 C1>
    {10.75} <music21.chord.Chord C1 C1>
    {12.75} <music21.chord.Chord C1 C1>
    {13.0} <music21.chord.Chord C1 C1>
    {13.25} <music21.chord.Chord C1 C1>
    {15.25} <music21.chord.Chord C1 C1>
    {17.25} <music21.chord.Chord C1 C1>
    {19.25} <music21.chord.Chord C1 C1>
    {19.5} <music21.chord.Chord C1 C1>
    {19.75} <music21.chord.Chord C1 C1>
    {21.75} <music21.chord.Chord C1 C1>
    {23.75} <music21.chord.Chord C1 C1>
    {25.75} <music21.chord.Chord C1 C1>
    {26.0} <music21.chord.Chord C1 C1>
    {26.25} <music21.chord.Chord C1 C1>
    {28.25} <music21.chord.Chord C1 C1>
    {28.5} <